In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
import pandas as pd
from nltk.util import ngrams
import os
from Model_List import model_paths
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from dotenv import load_dotenv, dotenv_values
import torch

# Downloading the stopwords and wordnet resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
from huggingface_hub import login
login(token = os.getenv('HUGGING_FACE'), add_to_git_credential=False)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\rohan\.cache\huggingface\token
Login successful


In [3]:
df = pd.read_csv('./descriptions.csv')
print(df.shape)

(52472, 3)


BERT

In [8]:
from transformers import BertTokenizer

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example text
sample_text = "The focused object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place."

# Tokenize using BERT tokenizer
encoded_input = tokenizer(sample_text, return_tensors='pt')

print(encoded_input)

{'input_ids': tensor([[  101,  1996,  4208,  4874,  1999,  1996,  3746,  2003,  1037,  2235,
          1010,  2317, 13297,  2007,  1037,  2630,  1998,  2317, 13478,  1012,
          2009,  2038,  2048,  5209,  2006,  2169,  3358,  1010,  1998,  2045,
          2024,  3645,  2006,  1996,  3903,  1012,  1996, 13297,  3544,  2000,
          2022, 17337,  1010,  2007,  2053,  2060,  5200,  2030,  4506,  2635,
          2173,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}


c:\Users\rohan\anaconda3\envs\three\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT-2

In [ ]:
# model_name = "openai-community/gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# def summarize_text_with_gpt2(text):
#     # Combine the instructional prompt with the text
#     prompt = "Briefly describe the text and don't mention about image, background, or any actions:\n" + text
    
#     # Encode the prompt to get input IDs
#     inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
#     # Generate the output using the model
#     outputs = model.generate(inputs, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2)
    
#     # Decode and return the generated text
#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary
# long_text = 'The focussed object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place.'
# summary = summarize_text_with_gpt2(long_text)
# print("Summary:", summary)

LLAMA

In [4]:
model_name = "meta-llama/Meta-Llama-3.1-8B"
# model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model.to('cuda')
# def get_summary(text):
#     prompt_text = "Summarize the following text into no more than three sentences: " + text
#     inputs = tokenizer(prompt_text, return_tensors="pt")
#     outputs = model.generate(inputs.input_ids, max_length=150, num_return_sequences=1)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Example usage
# long_text = "Here you insert the long, less meaningful text that you need summarized. This text might include multiple aspects such as historical data, technical details, and anecdotal evidence, but the summary should distill only the most essential information."
# summary = get_summary(long_text)
# print("Summary:", summary)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# def get_summary(text):
#     prompt_text = "Summarize the following text into no more than three sentences: " + text
#     inputs = tokenizer(prompt_text, return_tensors="pt")
#     outputs = model.generate(inputs.input_ids, max_length=150, num_return_sequences=1)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Example usage
# long_text = "The focussed object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place."
# summary = get_summary(long_text)
# print("Summary:", summary)

In [ ]:
# model_name = "openai-community/gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# def summarize_text_with_gpt2(text):
#     # Combine the instructional prompt with the text
#     prompt = get_prompt(text)
#     # Encode the prompt to get input IDs
#     inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=1024, truncation=True)
#     # Generate the output using the model
#     outputs = model.generate(inputs, max_length=500, num_return_sequences=1, no_repeat_ngram_size=2, pad_token_id=tokenizer.pad_token_id)
#     outputs = model.generate(
#     inputs,
#     max_length=500,
#     num_return_sequences=1,
#     num_beams=1,           # Use a single beam for determinism
#     no_repeat_ngram_size=2,
#     do_sample=False,        # Turn off sampling; use deterministic decoding
#     pad_token_id=tokenizer.pad_token_id
# )
#     # outputs = model.generate(inputs, max_length=500, num_return_sequences=1, pad_token_id=tokenizer.pad_token_id)
#     # Decode and return the generated text
#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#     # summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary
# long_text = 'The focussed object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place.'
# summary = summarize_text_with_gpt2(long_text)
# print(summary.split('\n')[-1])
# model_name = 'facebook/bart-large-cnn'
# tokenizer = BartTokenizer.from_pretrained(model_name)
# model = BartForConditionalGeneration.from_pretrained(model_name)
# def summarize_text(text):
#     # Encode the text input and get the generated summary
#     inputs = tokenizer.encode("summarize in one line" + text, return_tensors='pt', max_length=1024, truncation=True)
#     # summary_ids = model.generate(inputs, max_length=5000, min_length=4000, length_penalty=2.0, num_beams=4, early_stopping=True)
#     summary_ids = model.generate(inputs, max_length=500, num_beams=4, early_stopping=True)
#     summary = tokenizer.decode(summary_ids[-1], skip_special_tokens=True)

#     return summary

# # Your text paragraph
# text_paragraph = """
# The focussed object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place.
# """

# # Get the summary
# summary = summarize_text(text_paragraph)
# print(summary)
# from transformers import BartTokenizer, BartForConditionalGeneration

# # Initialize the tokenizer and model
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
# model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# # Few-shot examples followed by the new input
# prompt = """
# Summarize the sentence "I am going shopping because I don't have food in my fridge."
# Summary:
# """

# # Tokenize the prompt
# inputs = tokenizer([prompt], max_length=1024, return_tensors='pt', truncation=True)

# # Generate summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
# summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# print(summary)
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# # Initialize the tokenizer and model
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# model = T5ForConditionalGeneration.from_pretrained('t5-base')

# prompt = """summarize: The focussed object in the image is a small, white airplane with a blue and white fuselage. It has two engines on each wing, and there are windows on the sides. The airplane appears to be stationary, with no other objects or actions taking place."""
# # Tokenize the prompt, setting max_length appropriately
# inputs = tokenizer(prompt, return_tensors='pt')

# # Generate summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=7, min_length=20, max_length=40)
# summary = tokenizer.decode(summary_ids[0], skip_special_tokens=False)

# print(summary)

2D visualization of text embeddings

In [ ]:
# # Load the embeddings
# datafile_path = "./embedding.csv"
# df = pd.read_csv(datafile_path)
# subclasses = df['Subclass']
# # Convert to a list of lists of floats
# matrix = np.array(df['Embedding'].apply(literal_eval).to_list())

# # Create a t-SNE model and transform the data
# tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
# vis_dims = tsne.fit_transform(matrix)
# unique_subclasses = np.unique(subclasses)
# subclass_map = {label: idx for idx, label in enumerate(unique_subclasses)}
# color_labels = subclasses.map(subclass_map)  # Convert labels to integers
# # Normalize your labels to be between 0 and 1
# color_labels = (color_labels - color_labels.min()) / (color_labels.max() - color_labels.min())

# cmap = cm.viridis
# # Plot the results
# plt.figure(figsize=(12, 8))
# scatter = plt.scatter(vis_dims[:, 0], vis_dims[:, 1], alpha=0.5, cmap=cm.viridis, c=color_labels)
# plt.colorbar(scatter)  # Show color scale
# plt.title('2D visualization of Text Embeddings')
# plt.xlabel('Dimension 1')
# plt.ylabel('Dimension 2')
# plt.show()

3D Visualization of text embeddings

In [ ]:
# import pandas as pd
# import numpy as np
# from ast import literal_eval
# from sklearn.manifold import TSNE
# from matplotlib import pyplot as plt
# from matplotlib import cm
# from mpl_toolkits.mplot3d import Axes3D  # This is needed for 3D plotting

# # Load the embeddings
# datafile_path = "./embedding.csv"
# df = pd.read_csv(datafile_path)
# subclasses = df['Subclass']
# # Convert to a list of lists of floats
# matrix = np.array(df['Embedding'].apply(literal_eval).to_list())

# # Create a t-SNE model and transform the data
# tsne = TSNE(n_components=3, perplexity=15, random_state=42, init='random', learning_rate=200)
# vis_dims = tsne.fit_transform(matrix)

# # Retrieve unique subclasses and create a mapping
# unique_subclasses = np.unique(subclasses)
# subclass_map = {label: idx for idx, label in enumerate(unique_subclasses)}
# color_labels = subclasses.map(subclass_map)  # Convert labels to integers

# # Normalize your labels to be between 0 and 1
# color_labels = (color_labels - color_labels.min()) / (color_labels.max() - color_labels.min())

# cmap = cm.viridis

# # Plot the results in 3D
# fig = plt.figure(figsize=(12, 10))
# ax = fig.add_subplot(111, projection='3d')
# scatter = ax.scatter(vis_dims[:, 0], vis_dims[:, 1], vis_dims[:, 2], alpha=0.5, cmap=cm.viridis, c=color_labels)
# plt.colorbar(scatter)  # Show color scale
# ax.set_title('3D Visualization of Text Embeddings')
# ax.set_xlabel('Dimension 1')
# ax.set_ylabel('Dimension 2')
# ax.set_zlabel('Dimension 3')
# plt.show()